![cactus](http://cactuscode.org/global/images/cactuslogo.png)
# Compiling Cactus!
Step 1 is to download the code. Cactus uses a script named "GetComponents" to find and prepare all the source code modules that it needs for a given installation. The GetComponents script can be downloaded with a simple invocation of curl.

In [1]:
import os
os.environ["PATH"]="/opt/conda/envs/python2/bin:"+os.environ["PATH"]

In [2]:
!python --version

Python 2.7.13


In [3]:
%cd ~/
!curl -kLO https://raw.githubusercontent.com/gridaphobe/CRL/ET_2016_11/GetComponents

/home/sbrandt
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 99330  100 99330    0     0   407k      0 --:--:-- --:--:-- --:--:--  409k


Step 2 is to download your thornlist. In this tutorial, we are going to use Funwave, a collection of thorns
designed to simulate water waves using the Boussinesq equations.

In [4]:
!curl -kLO https://bitbucket.org/stevenrbrandt/cajunwave/raw/master/funwave_carpet.th

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3994  100  3994    0     0  12085      0 --:--:-- --:--:-- --:--:-- 12103


You can view a file in the notebook by using the "magic" command "%pycat filename". However, it tries to highlight
syntax as if the file is written in python. In those cases you can simply use "%cat filename." Unfortunately, unlike %pycat, %cat leaves the contents of the file on the screen.

Note that at the top of the file is "DEFINE_ROOT = CactusFW2". This means that Cactus, and all its thorns, will be checked out under that directory.

In [5]:
%pycat ~/funwave_carpet.th

Next we need to checkout the components listed in the thornlist. We do this with the GetComponents command.
Before we can execute it, however, we need to turn on its execute bit.

In [7]:
%cd ~/
!chmod a+x GetComponents
!echo no|./GetComponents --parallel funwave_carpet.th

/home/sbrandt
-----------------------------------------------------------------
  Checking out module: CONTRIBUTORS
      from repository: https://bitbucket.org/cactuscode/cactus.git
                 into: CactusFW2
                   as: flesh
-----------------------------------------------------------------
  Checking out module: COPYRIGHT
      from repository: https://bitbucket.org/cactuscode/cactus.git
                 into: CactusFW2
                   as: flesh
-----------------------------------------------------------------
  Checking out module: doc
      from repository: https://bitbucket.org/cactuscode/cactus.git
                 into: CactusFW2
                   as: flesh
-----------------------------------------------------------------
  Checking out module: lib
      from repository: https://bitbucket.org/cactuscode/cactus.git
                 into: CactusFW2
                   as: flesh
-----------------------------------------------------------------
  Checking out mo

-----------------------------------------------------------------
  Checking out module: Carpet/CarpetIOScalar
      from repository: https://bitbucket.org/eschnett/carpet.git
                 into: CactusFW2/arrangements
-----------------------------------------------------------------
  Checking out module: ExternalLibraries/MPI
      from repository: https://svn.cactuscode.org/projects/ExternalLibraries/MPI/trunk
                 into: CactusFW2/arrangements
-----------------------------------------------------------------
  Checking out module: Carpet/TestLoopControl
      from repository: https://bitbucket.org/eschnett/carpet.git
                 into: CactusFW2/arrangements
-----------------------------------------------------------------
  Checking out module: ExternalLibraries/HDF5
      from repository: https://svn.cactuscode.org/projects/ExternalLibraries/HDF5/trunk
                 into: CactusFW2/arrangements
-----------------------------------------------------------------

In [8]:
%cd ~/CactusFW2

/home/sbrandt/CactusFW2


<img src="http://simfactory.org/global/images/factory.png" width="200px">

# Simfactory
Cactus is normally built with a tool called Simfactory. Simfactory, in turn, will call make.
Before it can work, however, it needs to be configured. Please replace... my email address in the
command below with yours. The email address isn't sent anywhere, all it's used for is allowing
Cactus to send job change state notifications to you.

In [9]:
%cd ~/CactusFW2
!./simfactory/bin/sim setup-silent --setup-email=sbrandt@cct.lsu.edu 

/home/sbrandt/CactusFW2
Unknown machine name bf6aca18b8a6


Here we will define some necessary Simulation Factory defaults.


Determining local machine name: bf6aca18b8a6
Creating machine bf6aca18b8a6 from generic: machine bf6aca18b8a6 [/home/sbrandt/CactusFW2/repos/simfactory2/mdb/machines/bf6aca18b8a6.ini] created successfully

--------------------SUMMARY--------------------:


[default]
user            = sbrandt
email           = sbrandt@cct.lsu.edu
allocation      = NO_ALLOCATION


------------------END SUMMARY------------------:

Contents successfully written to /home/sbrandt/CactusFW2/repos/simfactory2/etc/defs.local.ini


At long last, we are ready to actually build Cactus. Cactus can often figure out what compilers and build
options to use automatically, but in some cases it is necessary to specify it by hand (you can do this by adding --optionlist=centos.cfg to the build command below). The file containing
this information is called the Option List. You might want to take a look at it.

This is the command to build Cactus using our thornlist. As written, it will build in parallel using two processes. That's what the -j option does.

In [10]:
#!rm -fr configs

In [11]:
!time ./simfactory/bin/sim build --mdbkey make 'make -j2' --thornlist=./repos/cajunwave/funwave_carpet.th | cat -

Using configuration: sim
Reconfiguring sim
Writing configuration to: /home/sbrandt/CactusFW2/configs/sim/OptionList
Cactus - version: 4.2.3
Reconfiguring sim.
Using configuration options from configure line
  Setting fds to '4,5 --'
End of options from configure line
Adding configuration options from '/home/sbrandt/CactusFW2/configs/sim/OptionList'...
  Setting VERSION to '2018-01-25'
  Setting CPP to 'cpp'
  Setting CC to 'gcc'
  Setting CXX to 'g++'
  Setting FPP to 'cpp'
  Setting F90 to 'gfortran'
  Setting CPPFLAGS to ''
  Setting FPPFLAGS to '-traditional'
  Setting CFLAGS to '-g -std=gnu99'
  Setting CXXFLAGS to '-g -std=gnu++0x'
  Setting F90FLAGS to '-g -fcray-pointer -ffixed-line-length-none'
  Setting LDFLAGS to '-rdynamic'
  Setting DEBUG to 'no'
  Setting CPP_DEBUG_FLAGS to ''
  Setting C_DEBUG_FLAGS to ''
  Setting CXX_DEBUG_FLAGS to ''
  Setting OPTIMISE to 'yes'
  Setting CPP_OPTIMISE_FLAGS to ''
  Setting C_OPTIMISE_FLAGS to '-O2'
  Setting CXX_OPTIMISE_FLAGS to '-O2'


checking for isinf in header math.h and library m... yes
checking for isnan in header math.h and library m... yes
checking for isnormal in header math.h and library m... yes
checking for signbit in header math.h and library m... yes
checking for mkstemp... yes
checking for va_copy... yes
creating cctk_Archdefs.h
creating make.arch.defn
creating cctk_Extradefs.h
creating make.extra.defn
updating cache ./config.cache
creating ./config.status
creating make.config.defn
creating make.config.deps
creating make.config.rules
creating cctk_Config.h
Determining number of fortran underscores...
Compiling test file with gfortran -g -fcray-pointer -ffixed-line-length-none ...
Lowercase - One trailing underscore
Compiling test file with gfortran -g -fcray-pointer -ffixed-line-length-none ...
Lowercase - One trailing underscore
Use make sim to build the configuration.
Complete rebuild required
Cleaning sim
Cleaning configuration sim
Deleting all built files in /home/sbrandt/CactusFW2/configs/sim
Upda

          0 Global, 59 Restricted, 2 Private parameters
   InitBase
          0 Global, 1 Restricted, 0 Private parameters
   InterpToArray
          0 Global, 93 Restricted, 0 Private parameters
   LocalInterp
          0 Global, 0 Restricted, 0 Private parameters
   LocalReduce
          0 Global, 0 Restricted, 0 Private parameters
   LoopControl
          0 Global, 0 Restricted, 20 Private parameters
   MPI
          0 Global, 0 Restricted, 0 Private parameters
   MoL
          0 Global, 9 Restricted, 26 Private parameters
   NaNChecker
          0 Global, 0 Restricted, 11 Private parameters
   SpaceMask
          0 Global, 1 Restricted, 0 Private parameters
   SphericalSurface
          0 Global, 15 Restricted, 9 Private parameters
   SymBase
          0 Global, 0 Restricted, 1 Private parameters
   SystemTopology
          0 Global, 0 Restricted, 2 Private parameters
   TestCarpetGridInfo
          0 Global, 0 Restricted, 2 Private parameters
   TestLoopControl
          0 Global,

COMPILING arrangements/AEIThorns/AEILocalInterp/src/Lagrange-maximum-degree/2d.cube.order1.smooth0.c
COMPILING arrangements/AEIThorns/AEILocalInterp/src/Lagrange-tensor-product/2d.cube.order3.smooth0.c
COMPILING arrangements/AEIThorns/AEILocalInterp/src/Lagrange-maximum-degree/2d.cube.order2.smooth0.c
COMPILING arrangements/AEIThorns/AEILocalInterp/src/Lagrange-tensor-product/2d.cube.order4.smooth0.c
COMPILING arrangements/AEIThorns/AEILocalInterp/src/Lagrange-maximum-degree/2d.cube.order3.smooth0.c
COMPILING arrangements/AEIThorns/AEILocalInterp/src/Lagrange-maximum-degree/2d.cube.order4.smooth0.c
COMPILING arrangements/AEIThorns/AEILocalInterp/src/Lagrange-tensor-product/3d.cube.order1.smooth0.c
COMPILING arrangements/AEIThorns/AEILocalInterp/src/Lagrange-maximum-degree/3d.cube.order1.smooth0.c
COMPILING arrangements/AEIThorns/AEILocalInterp/src/Lagrange-tensor-product/3d.cube.order2.smooth0.c
COMPILING arrangements/AEIThorns/AEILocalInterp/src/Lagrange-maximum-degree/3d.cube.order2.

COMPILING configs/sim/bindings/build/IOBasic/cctk_ThornBindings.c
Creating /home/sbrandt/CactusFW2/configs/sim/lib/libthorn_IOBasic.a
Checking status of thorn InitBase
COMPILING configs/sim/bindings/build/InitBase/cctk_ThornBindings.c
Creating /home/sbrandt/CactusFW2/configs/sim/lib/libthorn_InitBase.a
Checking status of thorn InterpToArray
COMPILING arrangements/CactusNumerical/InterpToArray/src/interp.c
COMPILING configs/sim/bindings/build/InterpToArray/cctk_ThornBindings.c
Creating /home/sbrandt/CactusFW2/configs/sim/lib/libthorn_InterpToArray.a
Checking status of thorn LocalInterp
COMPILING arrangements/CactusNumerical/LocalInterp/src/Startup.c
COMPILING arrangements/CactusNumerical/LocalInterp/src/Operator.c
COMPILING arrangements/CactusNumerical/LocalInterp/src/Interpolate.c
COMPILING configs/sim/bindings/build/LocalInterp/cctk_ThornBindings.c
Creating /home/sbrandt/CactusFW2/configs/sim/lib/libthorn_LocalInterp.a
Checking status of thorn LocalReduce
COMPILING arrangements/Cactus

...

COMPILING arrangements/CactusCoastal/Funwave/src/molregister.cc
COMPILING arrangements/CactusCoastal/Funwave/src/fluxes.cc
COMPILING arrangements/CactusCoastal/Funwave/src/sourceterms.cc
COMPILING arrangements/CactusCoastal/Funwave/src/sponge.cc
COMPILING arrangements/Carpet/CarpetLib/src/gdata.cc
COMPILING arrangements/CactusCoastal/Funwave/src/dispersion.cc
In file included from /home/sbrandt/CactusFW2/configs/sim/build/CarpetLib/gdata.cc:17:0:
/home/sbrandt/CactusFW2/configs/sim/build/CarpetLib/gdata.cc: In function 'ompi_datatype_t* mpi_datatype(const slab<T, D>&)':
/usr/lib/x86_64-linux-gnu/openmpi/include/mpi.h:1076:53: warning: 'ompi_mpi_ub' is deprecated: MPI_UB is deprecated in MPI-2.0 [-Wdeprecated-declarations]
 #define MPI_UB OMPI_PREDEFINED_GLOBAL(MPI_Datatype, ompi_mpi_ub)
                                                     ^
/usr/lib/x86_64-linux-gnu/openmpi/include/mpi.h:313:66: note: in definition of macro 'OMPI_PREDEFINED_GLOBAL'
 #define OMPI_PREDEFINED_GLOBAL(type,

COMPILING arrangements/Carpet/CarpetLib/src/startup_time.cc
COMPILING arrangements/Carpet/CarpetLib/src/th.cc
COMPILING arrangements/Carpet/CarpetLib/src/timestat.cc
COMPILING arrangements/Carpet/CarpetLib/src/vect.cc
COMPILING arrangements/Carpet/CarpetLib/src/copy_3d.cc
In file included from /home/sbrandt/CactusFW2/arrangements/Carpet/CarpetLib/src/dist.hh:12:0,
                 from /home/sbrandt/CactusFW2/arrangements/Carpet/CarpetLib/src/vect.hh:13,
                 from /home/sbrandt/CactusFW2/arrangements/Carpet/CarpetLib/src/bbox.hh:11,
                 from /home/sbrandt/CactusFW2/arrangements/Carpet/CarpetLib/src/bboxset1.hh:11,
                 from /home/sbrandt/CactusFW2/arrangements/Carpet/CarpetLib/src/bboxset.hh:6,
                 from /home/sbrandt/CactusFW2/configs/sim/build/CarpetLib/vect.cc:8:
/home/sbrandt/CactusFW2/configs/sim/build/CarpetLib/vect.cc: In member function 'ompi_datatype_t* vect<T, D>::mpi_datatype() const':
/usr/lib/x86_64-linux-gnu/openmpi/include

COMPILING arrangements/Carpet/CarpetLib/src/copy_4d.cc
COMPILING arrangements/Carpet/CarpetLib/src/interpolate_3d_2tl.cc
COMPILING arrangements/Carpet/CarpetLib/src/interpolate_3d_3tl.cc
COMPILING arrangements/Carpet/CarpetLib/src/interpolate_3d_4tl.cc
COMPILING arrangements/Carpet/CarpetLib/src/interpolate_3d_5tl.cc
COMPILING arrangements/Carpet/CarpetLib/src/interpolate_eno_3d_3tl.cc
COMPILING arrangements/Carpet/CarpetLib/src/restrict_3d_cc_rf2.cc
COMPILING arrangements/Carpet/CarpetLib/src/restrict_3d_cc_o3_rf2.cc
COMPILING arrangements/Carpet/CarpetLib/src/restrict_3d_cc_o5_rf2.cc
COMPILING arrangements/Carpet/CarpetLib/src/restrict_3d_rf2.cc
COMPILING arrangements/Carpet/CarpetLib/src/restrict_3d_stagger011.cc
COMPILING arrangements/Carpet/CarpetLib/src/restrict_3d_stagger101.cc
COMPILING arrangements/Carpet/CarpetLib/src/restrict_3d_stagger110.cc
COMPILING arrangements/Carpet/CarpetLib/src/restrict_3d_stagger111.cc
COMPILING arrangements/Carpet/CarpetLib/src/restrict_3d_vc_rf2.c

COMPILING configs/sim/bindings/build/CarpetIOScalar/cctk_ThornBindings.c
Creating /home/sbrandt/CactusFW2/configs/sim/lib/libthorn_CarpetIOScalar.a
COMPILING arrangements/Carpet/CarpetIOHDF5/src/Output.cc
COMPILING configs/sim/bindings/build/CarpetIOHDF5/cctk_ThornBindings.c
Checking status of thorn CarpetInterp
COMPILING arrangements/Carpet/CarpetInterp/src/interp.cc
COMPILING configs/sim/bindings/build/CarpetInterp/cctk_ThornBindings.c
Creating /home/sbrandt/CactusFW2/configs/sim/lib/libthorn_CarpetInterp.a
COMPILING arrangements/Carpet/CarpetIOHDF5/src/OutputSlice.cc
Checking status of thorn CarpetInterp2
COMPILING arrangements/Carpet/CarpetInterp2/src/fasterp.cc
/home/sbrandt/CactusFW2/configs/sim/build/CarpetIOHDF5/OutputSlice.cc: In instantiation of 'static int CarpetIOHDF5::IOHDF5<outdim>::WriteHDF5(const cGH*, hid_t&, hid_t&, std::vector<gdata*>, const bbox<int, 3>&, int, const vect<int, 3>&, const vect<int, outdim>&, int, int, int, int, int, int, CCTK_REAL8, const vect<double,

COMPILING configs/sim/bindings/build/CarpetRegrid/cctk_ThornBindings.c
Creating /home/sbrandt/CactusFW2/configs/sim/lib/libthorn_CarpetRegrid.a
COMPILING configs/sim/bindings/build/CarpetRegrid2/cctk_ThornBindings.c
Checking status of thorn CarpetSlab
COMPILING arrangements/Carpet/CarpetRegrid2/src/property.cc
COMPILING arrangements/Carpet/CarpetSlab/src/Get.cc
/home/sbrandt/CactusFW2/configs/sim/build/CarpetRegrid2/property.cc:211:2: warning: #warning "TODO: Correct this" [-Wcpp]
 #warning "TODO: Correct this"
  ^~~~~~~
COMPILING arrangements/Carpet/CarpetSlab/src/GetHyperslab.cc
COMPILING arrangements/Carpet/CarpetSlab/src/mapping.cc
COMPILING arrangements/Carpet/CarpetSlab/src/slab.cc
COMPILING arrangements/Carpet/CarpetRegrid2/src/regrid.cc
COMPILING configs/sim/bindings/build/CarpetSlab/cctk_ThornBindings.c
Creating /home/sbrandt/CactusFW2/configs/sim/lib/libthorn_CarpetSlab.a
Checking status of thorn TestCarpetGridInfo
COMPILING arrangements/Carpet/TestCarpetGridInfo/src/test.cc


The build command creates a configuration called "sim". It is found in the "configs/sim" directory. One of the files in this directory is the ThornList. It contains the list of thorns Cactus will compile. If you wish to add or remove a thorn from your configuration, you can do it by editing this file. However, by doing so you risk confusing yourself by forgetting what you've done. Proceed at your own risk!

In [12]:
%ls ~/CactusFW2/configs/sim

OptionList    ThornList  config-data/  lib/            scratch/
RunScript     bindings/  config-info   piraha/
SubmitScript  build/     datestamp.o   properties.ini


In [13]:
%pycat ~/CactusFW2/configs/sim/ThornList

The "OptionList" file contains all the configuration options (the things you saw in centos.cfg). Unlike the ThornList file, however, changing this file will have no effect. If you wish to change your configuration options without starting over from scratch, you should edit the file "configs/sim/config-data/make.config.defn."

In [16]:
%pycat ~/CactusFW2/configs/sim/config-data/make.config.defn

<h1>Running Cactus!</h1>

In [17]:
%cd ~/CactusFW2

/home/sbrandt/CactusFW2


Below we are going to run a simple Gaussian water wave over a flat seabed. We will use MPI and run on two processes. You can edit the parameter file below and hit shift-Enter to write it to disk. The special sequence "%%writefile filename" at the top makes this possible. Alternatively, you can load an existing file by putting the special sequence "%load filename" at the top of a cell and hitting shift-Enter.

In [18]:
%%writefile ~/CactusFW2/wave.par

#Reorder the parameters for easy comparison to the input.txt in example 3
ActiveThorns = "
  CoordBase FunWave FunwaveCoord CartGrid3D Carpet CarpetIOASCII
  CartGrid3D IOUtil CarpetIOBasic CarpetSlab Boundary SymBase MoL
  CarpetReduce LocalReduce InitBase CarpetLib LoopControl Tridiagonal
  CarpetIOScalar "

#----------------------------------------------------
# Flesh and CCTK parameters
#----------------------------------------------------

# flesh
Cactus::cctk_run_title = "Test Run"
Cactus::cctk_show_schedule = "yes"
Cactus::cctk_itlast = 300
Cactus::allow_mixeddim_gfs = "yes"

# CartGrid3D
CartGrid3D::type = "coordbase"
CartGrid3D::avoid_origin = "no"
CoordBase::domainsize = "minmax"
CoordBase::spacing    = "gridspacing"
CoordBase::xmin =  0
CoordBase::xmax =  30
CoordBase::ymin =  0
CoordBase::ymax =  30
CoordBase::zmin =  0.0
CoordBase::zmax =  0.0
CoordBase::dx   =  0.25
CoordBase::dy   =  0.25

CoordBase::boundary_size_x_lower     = 3
CoordBase::boundary_size_x_upper     = 3
CoordBase::boundary_size_y_lower     = 3
CoordBase::boundary_size_y_upper     = 3
CoordBase::boundary_size_z_lower     = 0
CoordBase::boundary_size_z_upper     = 0
CoordBase::boundary_shiftout_x_lower = 1
CoordBase::boundary_shiftout_x_upper = 1
CoordBase::boundary_shiftout_y_lower = 1
CoordBase::boundary_shiftout_y_upper = 1
CoordBase::boundary_shiftout_z_lower = 1
CoordBase::boundary_shiftout_z_upper = 1

# Carpet
Carpet::domain_from_coordbase = "yes"
Carpet::ghost_size_x = 3
Carpet::ghost_size_y = 3
Carpet::ghost_size_z = 1
carpet::adaptive_stepsize = yes

# MoL
MoL::ODE_Method = "RK3"
MoL::disable_prolongation        = "yes"

# the output dir will be named after the parameter file name
IO::out_dir = $parfile
IO::out_fileinfo="none"
IOBasic::outInfo_every = 1
IOBasic::outInfo_vars = "FunWave::eta FunWave::u FunWave::v"

#IOASCII::out1D_every = 1
#IOASCII::out1d_vars = "FunWave::eta Funwave::depth"
CarpetIOASCII::compact_format = false
IOASCII::out2D_every = 30
IOASCII::out2D_xyplane_z = 0
IOASCII::out2D_vars = "FunWave::eta FunWave::u FunWave::v"
IOASCII::out2D_xz = "no"
IOASCII::out2D_yz = "no"
IOASCII::output_ghost_points = "no"

IOScalar::outScalar_every = 1
IOScalar::outScalar_vars = "FunWave::eta FunWave::u FunWave::v"

#& = "Funwave::eta"

#----------------------------------------------------
# Funwave parameters
#----------------------------------------------------

# Funwave depth 
FunWave::depth_file_offset_x = 3
FunWave::depth_file_offset_y = 3
FunWave::depth_type = "flat"
FunWave::depth_format = "ele"
FunWave::depth_file = "/tmp/__depth__.txt"
FunWave::depth_flat = 0.8
#Funwave::test_depth_shore_x = 80
#Funwave::test_depth_island_x = 40
#Funwave::test_depth_island_y = 40
FunWave::depth_xslp = 10.0
FunWave::depth_slope = 0.05
FunWave::dt_size = 0
Funwave::generate_test_depth_data = true
Funwave::num_wave_components = 1
Funwave::wave_component_file = "/home/sbrandt/workspace/shi_funwave/example_2/fft/wavemk_per_amp_pha.txt"
Funwave::peak_period = 1

# import
Funwave::time_ramp = 1.0
Funwave::delta_wk = 0.5
Funwave::dep_wk = 0.45
Funwave::xc_wk = 3.0
Funwave::ywidth_wk = 10000.0
Funwave::tperiod = 1.0
Funwave::amp_wk = 0.0232
Funwave::theta_wk = 0.0
Funwave::freqpeak = 0.2
Funwave::freqmin = 0.1
Funwave::freqmax = 0.4
Funwave::hmo = 1.0
Funwave::gammatma = 5.0
Funwave::thetapeak = 10.0
Funwave::sigma_theta = 15.0

# Funwave wind forcing
Funwave::wind_force = false
Funwave::use_wind_mask = false
Funwave::num_time_wind_data = 2
Funwave::timewind[0] = 0
Funwave::wu[0] = 25
Funwave::wv[0] = 50
Funwave::timewind[1] = 1000
Funwave::wu[1] = 100
Funwave::wv[1] = 100
Funwave::boundary = funwave

# Funwave wave maker
FunWave::wavemaker_type = "ini_gau"
FunWave::xc = 26.5
FunWave::yc = 26.9
FunWave::amp =  2.0
FunWave::wid =  1
Funwave::wdep = 0.78
Funwave::xwavemaker = 25.0

# Funwave sponge 
FunWave::sponge_on = false
FunWave::sponge_west_width = 2.0
FunWave::sponge_east_width = 2.0
FunWave::sponge_north_width = 0.0
FunWave::sponge_south_width = 0.0
FunWave::sponge_decay_rate = 0.9
FunWave::sponge_damping_magnitude = 5.0

# Funwave dispersion (example 3 enables dispersion)
FunWave::dispersion_on = "true"
FunWave::gamma1 = 1.0
FunWave::gamma2 = 1.0
FunWave::gamma3 = 1.0
FunWave::beta_ref = -0.531
FunWave::swe_eta_dep = 0.80
FunWave::cd = 0.0

# Funwave numerics (MoL parameter controls time integration scheme)
FunWave::reconstruction_scheme = "fourth"
FunWave::riemann_solver = "HLLC"
FunWave::dtfac = 0.5
FunWave::froudecap = 10.0
FunWave::mindepth = 0.001
FunWave::mindepthfrc = 0.001
FunWave::enable_masks = "true"
Funwave::estimate_dt_on = "true"

FunwaveCoord::spherical_coordinates = false

ActiveThorns = "CarpetIOHDF5"
IOHDF5::out2D_xyplane_z = 0 
IOHDF5::out2D_every = 10
IOHDF5::out2D_vars = " 
  FunWave::eta
  FunWave::u
  FunWave::v
  Grid::Coordinates{out_every=1000000000}
"
IOHDF5::out2D_xz = no
IOHDF5::out2D_yz = no

Writing /home/sbrandt/CactusFW2/wave.par


This next cell deletes our simulation in case we want to throw it away and start over again for some reason.

In [19]:
!rm -fr ~/simulations/wave

At long last, we are ready run Cactus. This configuration specifies running on two threads, with 1 thread per process. To execute this command, Cactus uses a "RunScript" stored in configs/sim/RunScript. You might want to take a look at it. Identifiers sandwiched between @ symbols get replaced by Simfactory prior to execution.

In [20]:
!cat ~/CactusFW2/configs/sim/RunScript

#!/bin/sh

echo "Preparing:"
set -x                          # Output commands
set -e                          # Abort on errors

cd @RUNDIR@-active

echo "Checking:"
pwd
hostname
date

echo "Environment:"
export CACTUS_NUM_PROCS=@NUM_PROCS@
export CACTUS_NUM_THREADS=@NUM_THREADS@
export GMON_OUT_PREFIX=gmon.out
export OMP_NUM_THREADS=@NUM_THREADS@
env | sort > SIMFACTORY/ENVIRONMENT

echo "Starting:"
export CACTUS_STARTTIME=$(date +%s)

if [ ${CACTUS_NUM_PROCS} = 1 ]; then
    if [ @RUNDEBUG@ -eq 0 ]; then
   	  @EXECUTABLE@ -L 3 @PARFILE@
    else
   	  gdb --args @EXECUTABLE@ -L 3 @PARFILE@
    fi
else
	mpirun -np @NUM_PROCS@ @EXECUTABLE@ -L 3 @PARFILE@
fi

echo "Stopping:"
date
echo "Done."


Enough already! Let's run Cactus!

In [21]:
%cd ~/CactusFW2
!./simfactory/bin/sim create-run --procs 2 --num-threads 1 wave.par

/home/sbrandt/CactusFW2
Parameter file: /home/sbrandt/CactusFW2/wave.par
Skeleton Created
Job directory: "/home/sbrandt/simulations/wave"
Executable: "/home/sbrandt/CactusFW2/exe/cactus_sim"
Option list: "/home/sbrandt/simulations/wave/SIMFACTORY/cfg/OptionList"
Submit script: "/home/sbrandt/simulations/wave/SIMFACTORY/run/SubmitScript"
Run script: "/home/sbrandt/simulations/wave/SIMFACTORY/run/RunScript"
Parameter file: "/home/sbrandt/simulations/wave/SIMFACTORY/par/wave.par"
Simulation name: wave
Assigned restart id: 0 
Running simulation wave
Preparing:
+ set -e
+ cd /home/sbrandt/simulations/wave/output-0000-active
+ echo Checking:
Checking:
+ pwd
/home/sbrandt/simulations/wave/output-0000-active
+ hostname
bf6aca18b8a6
+ date
Thu Sep  6 14:14:53 UTC 2018
+ echo Environment:
Environment:
+ export CACTUS_NUM_PROCS=2
+ export CACTUS_NUM_THREADS=1
+ export GMON_OUT_PREFIX=gmon.out
+ export OMP_NUM_THREADS=1
+ env
+ sort
+ echo Starting:
Starting:
+ date +%s
+ export CACTUS_STARTTIME=1

INFO (CycleClock): Calibrated CycleClock: 0.454522 ns per clock tick (2.20011 GHz)
  ____       _          __        __              
 / ___|__ _ (_)_   _ _ _\ \      / /_ ___   _____ 
| |   / _` || | | | | '_ \ \ /\ / / _` \ \ / / _ \
| |__| (_| || | |_| | | | \ V  V / (_| |\ V /  __/
 \____\__,_|/ |\__,_|_| |_|\_/\_/ \__,_| \_/ \___|
          |__/                                    
INFO (Vectors): Using vector size 2 for architecture SSE2 (64-bit precision)
--------------------------------------------------------------------------------
AMR driver provided by Carpet
--------------------------------------------------------------------------------
AMR info I/O provided by CarpetIOBasic
--------------------------------------------------------------------------------
AMR 0D ASCII I/O provided by CarpetIOASCII
--------------------------------------------------------------------------------
AMR 1D ASCII I/O provided by CarpetIOASCII
-------------------------------------------------------

        3     1.048 |   -0.7827099    0.7827247 |   -0.7908120    0.7945910 | -2.204493e-13    1.7406879
        4     1.071 |   -0.9871958    0.9871304 |   -1.0078458    1.0085949 | -8.400691e-14    1.5989647
        5     1.094 |   -1.1744423    1.1742063 |   -1.1909363    1.1836515 | -1.710410e-15    1.4476202
        6     1.117 |   -1.3292681    1.3289617 |   -1.3316063    1.3201955 | -1.152414e-14    1.2961226
        7     1.140 |   -1.4464243    1.4460941 |   -1.4436961    1.4892265 | -7.675744e-14    1.1507342
        8     1.162 |   -1.5469224    1.5458777 |   -1.6224632    1.6048745 | -9.317621e-16    1.0141344
        9     1.185 |   -1.6090563    1.6066068 |   -1.7050094    1.6735034 | -3.912694e-15    0.8880859
       10     1.208 |   -1.6386679    1.6345209 |   -1.7530102    1.7107463 | -4.626260e-15    0.7717948
       11     1.230 |   -1.8563836    1.8480030 |   -1.7674885    1.7295325 | -2.510327e-14    0.6754319
       12     1.253 |   -1.6739634    1.6693377 |   -1.

       70     2.615 |   -1.4642912    0.2767483 |   -1.4057981    0.2590229 |   -0.2137260    0.5887379
       71     2.640 |   -1.4474711    0.2761392 |   -1.3962712    0.2583219 |   -0.1941274    0.5831460
       72     2.664 |   -1.4513894    0.2742674 |   -1.4019325    0.2548907 |   -0.1721779    0.5784937
       73     2.688 |   -1.4414570    0.2710084 |   -1.3995690    0.2490426 |   -0.1655913    0.5714285
       74     2.713 |   -1.4288063    0.2647984 |   -1.3864879    0.2420549 |   -0.1676511    0.5688352
       75     2.738 |   -1.4283104    0.2554700 |   -1.3908008    0.2340046 |   -0.1698992    0.5647703
       76     2.762 |   -1.4137836    0.2437063 |   -1.3822609    0.2265682 |   -0.1718507    0.5740248
       77     2.787 |   -1.4091300    0.2367836 |   -1.3767554    0.2179821 |   -0.1737959    0.5916501
       78     2.812 |   -1.4046636    0.2274053 |   -1.3758512    0.2130156 |   -0.1766506    0.6169808
       79     2.836 |   -1.3859611    0.2155210 |   -1.3619419  

      135     4.244 |   -1.0714781    0.2157427 |   -1.0731768    0.2032334 |   -0.3671937    0.5053027
      136     4.272 |   -1.0583377    0.2157335 |   -1.0750110    0.2041498 |   -0.3673188    0.4955046
      137     4.300 |   -1.0608795    0.2156183 |   -1.0674998    0.2167764 |   -0.3671624    0.4946664
      138     4.328 |   -1.0548312    0.2188421 |   -1.0619842    0.2287137 |   -0.3667662    0.4851986
      139     4.356 |   -1.0463378    0.2180459 |   -1.0607194    0.2396080 |   -0.3661865    0.4839073
-------------------------------------------------------------------------------------------------------
Iteration      Time |                FUNWAVE::u |                FUNWAVE::v |              FUNWAVE::eta
                    |      minimum      maximum |      minimum      maximum |      minimum      maximum
-------------------------------------------------------------------------------------------------------
      140     4.384 |   -1.0467500    0.2169481 |   -1.0500058  

      201     6.149 |   -0.8533410    0.3717816 |   -0.8152867    0.4101634 |   -0.1750078    0.3320730
      202     6.180 |   -0.8535028    0.3681777 |   -0.8146050    0.4101118 |   -0.1751810    0.3311869
      203     6.210 |   -0.8442866    0.3647292 |   -0.8119215    0.4102397 |   -0.1756846    0.3295971
      204     6.240 |   -0.8425213    0.3612682 |   -0.8052784    0.4114481 |   -0.1757903    0.3285272
      205     6.271 |   -0.8400223    0.3594254 |   -0.8062386    0.4125626 |   -0.1758510    0.3263323
      206     6.301 |   -0.8282626    0.3601502 |   -0.8010470    0.4150443 |   -0.1762802    0.3253608
      207     6.332 |   -0.8316114    0.3606417 |   -0.7985980    0.4163994 |   -0.1765009    0.3236009
      208     6.362 |   -0.8263956    0.3617883 |   -0.7971293    0.4196593 |   -0.1764601    0.3225128
      209     6.393 |   -0.8192261    0.3630934 |   -0.7894353    0.4223273 |   -0.1769458    0.3202863
      210     6.424 |   -0.8197248    0.3634221 |   -0.7909435  

      269     8.302 |   -0.6756800    0.4045318 |   -0.6554061    0.4411120 |   -0.1571127    0.2206535
      270     8.334 |   -0.6711322    0.4046642 |   -0.6501599    0.4390332 |   -0.1563702    0.2209410
      271     8.367 |   -0.6710783    0.4044379 |   -0.6510131    0.4387896 |   -0.1558859    0.2205042
      272     8.400 |   -0.6706263    0.4032953 |   -0.6504622    0.4380201 |   -0.1553208    0.2189918
      273     8.432 |   -0.6644033    0.4028924 |   -0.6438784    0.4362545 |   -0.1545145    0.2199283
      274     8.465 |   -0.6678908    0.4028691 |   -0.6477349    0.4349431 |   -0.1541124    0.2190905
      275     8.497 |   -0.6647396    0.4019592 |   -0.6447659    0.4344681 |   -0.1535710    0.2188659
      276     8.530 |   -0.6627175    0.4007857 |   -0.6422107    0.4330321 |   -0.1528491    0.2185265
      277     8.563 |   -0.6636459    0.4009477 |   -0.6438056    0.4310260 |   -0.1520524    0.2170415
      278     8.595 |   -0.6576792    0.4002606 |   -0.6384055  

Data can be found in this directory. Using the next couple of commands, we will browse it.

In [22]:
%cd ~/simulations/wave/output-0000/wave

/home/sbrandt/simulations/wave/output-0000/wave


In [23]:
%ls *.asc

eta.average.asc   eta.norm_inf.asc  u.minimum.asc   v.inorm1.asc
eta.count.asc     eta.sum.asc       u.norm1.asc     v.inorm2.asc
eta.iaverage.asc  eta.xy.asc        u.norm2.asc     v.maximum.asc
eta.icount.asc    u.average.asc     u.norm_inf.asc  v.minimum.asc
eta.inorm1.asc    u.count.asc       u.sum.asc       v.norm1.asc
eta.inorm2.asc    u.iaverage.asc    u.xy.asc        v.norm2.asc
eta.maximum.asc   u.icount.asc      v.average.asc   v.norm_inf.asc
eta.minimum.asc   u.inorm1.asc      v.count.asc     v.sum.asc
eta.norm1.asc     u.inorm2.asc      v.iaverage.asc  v.xy.asc
eta.norm2.asc     u.maximum.asc     v.icount.asc


In [24]:
# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

The top of %pycat command showed us what the columns mean:
HYDROBASE::rho (hydrobase-rho)
* 1 iteration
* 2 time - how much time has passed in the simulation
* 3 the data, in this case the variable rho

Once we know all this, it is straightforward to plot the data.

In [25]:
lin_data = np.genfromtxt("eta.maximum.asc")

In [ ]:
lin_data

In [ ]:
plt.plot(lin_data[:,1],lin_data[:,2])

Python knows how to read regularly formatted text
files that use the # character for comments. Fortunately,
that's what Cactus produces in its asc files.

In [ ]:
file_data = np.genfromtxt("eta.xy.asc")

In [ ]:
file_data

In [ ]:
import matplotlib.cm as cm
# https://matplotlib.org/examples/color/colormaps_reference.html
cmap = cm.gist_rainbow

In [ ]:
sets = np.unique(file_data[:,0])
width = 8
height = 4
print("sets=",sets)
mn, mx = np.min(file_data[:,12]),np.max(file_data[:,12])
for which in sets: 
    print("which=",which)
    g = file_data[file_data[:,0]==which,:]
    x = g[:,5]
    y = g[:,6]
    z = g[:,12]
    zi = z.reshape(len(np.unique(y)),len(np.unique(x)))
    print('min/max=',np.min(zi),np.max(zi))
    plt.figure(figsize=(width, height))
    plt.imshow(zi[::-1,:],cmap,clim=(mn,mx))
    plt.show()

In [ ]:
%ls *.h5

<h2>Plotting HDF5 Data</h2>
HDF5 (Hierarchical Data Format 5) is a portable binary data format. As such, it is far more efficient to read and
write than ascii formats, and it is probably what you should normally use. Here, you can see how to read and display
the data.

In [ ]:
import h5py

In [ ]:
f5 = h5py.File("u.xy.h5")
for nm in f5:
    if not hasattr(f5[nm],"shape"):
        continue
    print("nm=",nm)
    d=np.copy(f5[nm])
    plt.figure()
    plt.imshow(d[::-1,:])
    plt.show()

Unfortunately, each set only has one component of the plot, i.e. the part belonging to one processor. To fix this, we'll
collect data sets belonging to an iteration and display them all together. In order to make this happen, we'll need the
x and y values for each component of the grid.

In [ ]:
import re

In [ ]:
f5x = h5py.File("x.xy.h5")
f5y = h5py.File("y.xy.h5")
x_coords = {}
y_coords = {}
for nm in f5x:
    print(nm)
    m = re.search(r'rl=.*c=\d+',nm)
    if m:
        k = m.group(0)
        x_coords[k]=np.copy(f5x[nm])
for nm in f5y:
    m = re.search(r'rl=.*c=\d+',nm)
    if m:
        k = m.group(0)
        y_coords[k]=np.copy(f5y[nm])

In [ ]:
f5 = h5py.File("u.xy.h5")
mn,mx = None,None

# Compute the min and max
for nm in f5:
    if not hasattr(f5[nm],"shape"):
        continue
    d5 = np.copy(f5[nm])
    tmin = np.min(d5)
    tmax = np.max(d5)
    if mn == None:
        mn,mx = tmin,tmax
    else:
        if tmin < mn:
            mn = tmin
        if tmax > mx:
            mx = tmax
            
# Collect all the pieces into the d5_tl dictionary
d5_tl = {}            
for nm in f5:
    if not hasattr(f5[nm],"shape"):
        continue
    # Parse the string nm...
    m = re.search(r'it=(\d+)\s+tl=\d+\s+(rl=(\d+)\s+c=(\d+))',nm)
    # group(1) is the iteration number
    # group(2) is "rl={number} c={number}"
    # group(3) is the number in "rl={number}"
    # group(4) is the number in "c={number}"
    grid = int(m.group(1))
    comp = int(m.group(4))
    k = m.group(2)
    if grid in d5_tl:
        d5_tl[grid]["x"] += [x_coords[k]] # append to the x array
        d5_tl[grid]["y"] += [y_coords[k]] # append to the y array
        d5_tl[grid]["D"] += [f5[nm]] # append to the data array
    else:
        d5_tl[grid] = {
            "x":[x_coords[k]],
            "y":[y_coords[k]],
            "D":[f5[nm]]
        }

# Sort the keys so that we display time levels in order
def keysetf(d):
    a = [] # create an empty list
    for k in d: # for each key in d
        a.append(k) # append it to the list
    return a
kys = keysetf(d5_tl.keys())
kys.sort()

# Show the figures, combing data from the same time level
for index in kys:
    data = d5_tl[index]
    print("iteration=",index)
    plt.figure() # put this before the plots you wish to combine
    plt.pcolor(data["x"][0],data["y"][0],data["D"][0],vmin=mn,vmax=mx)
    plt.pcolor(data["x"][1],data["y"][1],data["D"][1],vmin=mn,vmax=mx)
    plt.show() # show the plot.

<h3>Questions and Exercises:</h3>

* Run the above simulation using a single process instead of two. Do the plotting routines work? What changes did you have to make. What would you need to do to make it work with 3?
* Run the code at 1/2 the resolution.
* Position the Guassian wave at a different place on the grid.
* If you wanted to change the compiler or a compiler flag, how would you go about doing that?
* If you wanted to add another thorn to the list of thorns to compile, how would you go about doing that?
* If you wanted to create a thornlist that would check out Cactus under the Foo directory instead of the CactusFW2 directory, how would you do it?

<table><tr><td>This work sponsored by NSF grants <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1550551"> OAC 1550551</a> and <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1539567"> CCF 1539567</a></td><td><img src="https://www.nsf.gov/awardsearch/images/common/nsf_logo_bottom.png"></tr></table>